# Opening a Mexican restaurant in Melbourne

## Introduction

A key issue when intending to open a restaurant of any kind is its physical location. Although there are many means to selecting a location, one option is to find an area that fits the type of cuisine that one intends to offer. Fit, here, could mean that either there are already a number of similar restaurants in the area speaking for an active customer base (herding effect), or that there are many other restaurants (cuisines) available but none of the particular cuisine one intends to offer, which could be considered a gap in the market.

The intention of this application is to use location data available through the Foursquare API to cluster suburbs (i.e. neighborhoods) in the City of Melbourne, Australia according to the top rated restaurants (cuisines) in each suburb to then help with the decision for a new restaurant location. For this project, I am using a Mexican restaurant as an example case. A location (or cluster of locations) would be seen as favourable either if there are other Mexican restaurants among the top rated ones (meaning customers already know this suburb for Mexican food), or if there are no Mexican restaurants among the top rated ones but the top rated ones create a profile that "matches" with Mexican cuisine (i.e. other South/Central American restaurants). Conversely, a suburb where all top rated restaurants are, for example, Chinese could be seen as not-favouring other types of cuisines.

I am aware that the notion of what is considered favourable or not favourable for a given restaurant is somewhat subjective, but for the sake of this assignment it should sufficient. Eventually, but this goes beyond the scope of this assignment, Foursquare's data could be linked with additional data, such as footfall, availability of parking and public transport, resident profiles, restaurant performance data etc., to create a more comprehensive basis for making location decisions.

# Data

The application builds on Foursquare's location data for restaurants and the types of cuisines they offer. To create a profile for each suburb, it uses the top 10 rated restaurants in each suburb and clusters them into seven groups (given about 200 postcodes, seven clusters seem more reasonable than five). I intend to use the second level of data under the "food" category (within the venue data) which classifies restaurants by their cuisine (country), matched with location data for the individual suburbs (collated by postcode) in Melbourne. Data on Melbourne's suburbs will be scraped from the Wikipedia page (https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs) and then merged with geospatial location data which I found here (http://www.corra.com.au/australian-postcode-location-data/).

## Methodology

### Scraping Melbourne suburb data off Wikipedia and cleaning up

After importing all required libraries (numpy, pandas, geocoders nominatim, json, matplotlib, kmeans, folium), I use BeautifulSoup to scrape the Melbourne suburb names and postcodes from the Wikipedia page, and the find function to find the right table to save the related data. I then create an empty dataframe with the nine columns equivalent to the table on Wikipedia and loop through the table while appending the values to the newly created dataframe. I then name all the columns as per the previous table and drop the ones I will not need (all except Postcode and Suburb). When Postcodes apply to more than one suburb, I am joining them under the same postcode. Because the data was scraped with the suburb in the first column and the postcode in the second, I am also switching the columns around. This results in a dataframe with 276 rows (unique postcodes).

Now it's time to add the latitude and longitude values for each postcode from the Corra website I found earlier. Because the CSV file is zipped, I downloaded it and then added it to my assets in Watson Studio. I only retained the postcode and lat/lon values, read the file into a new dataframe (df_geo), renamed the columns to match the first dataframe, and merged the two using the postcode as index. Because the geodata included separate coordinates for suburbs that share the same postcode, I dropped duplicates (on postcode), retaining the first entry.

### Creating map of Melbourne
With the postcode data sorted, I'm moving on to getting the coordinated for Melbourne and creating the map. I'm getting the map from Nominatim and am using folium to create markets that correspond with the locations in the combined "melbourne" dataframe.

### Getting top rated restaurants in each suburb
Next, I'm turning to Foursquare, first including my credentials and then defining a function that explores the respective postcodes (i.e. returns the top rated venues, limit=100). To only return "food" venues, I've included Foursquare's categoryID in the API url request. Running the function with the values from the previous "melbourne" dataframe produces a new "melbourne_venues" dataframe with 2392 rows and 7 columns. After checking how many restaurants were retuned for each postcode, I'm using onehot encoding to create dummies which denominate the existence, or not, of a particular type of restaurant in a particular suburb. Next I'm using the mean of the frequency of occurrence of each restaurant type to group the rows by suburb, resulting in a grouped dataframe (melbourne_grouped).

Using a new function, I'm sorting the top rated restaurants in each neighborhood in descending order and then create a new dataframe (suburbs_restaurants_sorted) which includes the top 10 restaurants (of food places) in each suburb. With this in place, I can move on to clustering the suburbs based on the restaurant information.

### Clustering suburbs
With a list of 197 suburbs, I've decided to use seven clusters to more comprehensively describe the food differences in each cluster. I'm using the grouped dataset that I prepared before. With the clustering done, I need to merge the grouped dataframe with the previous melbourne dataframe to include the latitude and longitude for the suburbs and to include the cluster labels. The resulting dataframe lists each postcode with according suburbs, cluster, and the ten most common restaurant types (cuisines).

Before examining at the results, I use Folium to visualize the clusters. To examine each cluster, I display, from the final merged dataframe, all postcodes/suburb with the according cluster label (0 for cluster 1, 1 for cluster 2, etc.). The clusters are described in the section below.

## Results

The seven clusters can be described as follows:
### Cluster 1: food heaven (everything from fine dining to food trucks)
Cluster 1 includes a large number of suburbs, all of which seem to have an abundant collection of cuisines. The respective lists of top restaurants include everything from fine dining places, over cafes and snack places, to fast food and food trucks. Notably, this is also the cluster with the highest density of Mexican restaurants among the top 10. Further sub-clustering of the suburbs that are part of this cluster would be helpful to single out those that do currently not have a Mexican restaurant in the top 10 listed venues.

### Cluster 2: fast food in the suburbs
Cluster 2 mainly includes industrial suburbs on the fringe of what might be considered Inner Melbourne. The top rated restaurants in these suburbs are almost exclusively fast food places (pizza, falafel, fish and chips, etc.) which are likely in the vicinity of the businesses operating there, providing quick lunch options to employees. Given the lack of any Mexican restaurants in this cluster, these suburbs seem unlikely locations unless one would consider a cheap and cheerful version, such as a taco truck. A proper restaurant is unlikely to find sufficient demand.

### Cluster 3 and cluster 6: you're lucky if you find anything but fish and chips
Clusters 3 and 6 are both relatively small and their list of top restaurants almost exclusively consists of pizza and donut shops, a food court, dumplings, and fish and chips. Suburbs included in these clusters are on the very fringes of Outer Melbourne and my assumption would be that you should consider yourself lucky if you get anything that is not deep fried after 3pm. For the same reasons as outlined in cluster 2, these suburbs most likely won't sustain a new Mexican restaurant.

### Cluster 4: Cafes and snacks, but also some foodie gems
Cluster 4 comprises suburbs on the fringe of Inner Melbourne, but, unlike those in cluster 2, there appear to be some more sophisticated options here and there that go beyond the usual fast food places. Specifically, all suburbs have a cafe listed as top food venue speaking for a lively breakfast culture. Beyond the food courts also found in cluster 2, most suburbs in cluster 4 include an Eastern European, Ethiopian, or Japanese restaurant, which might be considered gems in an otherwise somewhat bleak culinary landscape. Cluster 4 might provide interesting locations for a Mexican restaurant to be considered, if it can be positioned as one of these "hidden gems". At this point, the top 10 across suburbs in cluster 4 do not include a Mexican restaurant.

### Cluster 5: Inner city snacks
Similar to cluster 1, cluster 5 includes a relatively large number of suburbs, many of which are inner city locations, with a wide variety of cuisines. It appears, however, that the restaurants in cluster 5 are somewhat more "snack-oriented", with many cafes and bakeries, fried chicken, burger, sandwich, and sushi places. Although there are also many fast food places in cluster 5, it also contains a number of restaurants that could be considered finer dining. There are also a number of Mexican restaurants among the top 10, but fewer than were included in cluster 1. Building on the previously outlined results, suburbs in cluster 5 should definitely be considered for opening a Mexican restaurant, specifically if it is intended to be snack-oriented (e.g. tacos). The more inner city location of suburbs in this cluster should provide sufficient patronage to sustain a restaurant, even if the largest number of customers are likely to visit only for lunch.

### Cluster 7: pizza, burgers, and dumplings
The last cluster resembles cluster 4, but the included suburbs are a bit further out of the city than those in cluster 4. While there seems to be a focus on cafes, snacks, and pizza, the cluster also includes a number of more specialised restaurants that were not listed in the top 10 in other clusters, specifically some Spanish, Greek, Thai, Portuguese, as well as gastropubs which have not been listed highly before. There are, however, no Mexican restaurants included in the top 10 in this cluster. Following a similar argument as before, suburbs in cluster 7 might provide an ideal location for a Mexican restaurant in that customers appear to appreciate a variety of different cuisines (including Spanish and Portuguese) but currently do not have a Mexican option to choose. At the same time, there appears to be an appetite for anything ranging from fast food to fine dining, which would allow for different types of Mexican food, i.e. either low-key tacos or a sit-down restaurant with more sophisticated options.

## Discussion

Since I have already included my observations and some suggestions regarding the clusters' suitability for a Mexican restaurant in the results section, I will use this part mainly to provide a summary of the results and to comment on the usefulness of the analysis in its current form.

As outlined above, I would see clusters 1, 5, and 7 as most likely candidates to further investigate a suitable location for a Mexican restaurant, somewhat depending on the type of venue that is imagined (i.e. taco truck vs sit-down dining). These clusters are either already packed with comparable restaurants, meaning they should provide a good location for a new restaurant (given the quality is up to standard), or they do offer a broad variety of cuisine choices but do not currently have a Mexican restaurant listed in the top 10. Either should provide good chances for success, but further analysis and an expansion of the approach used here is definitely required.

Specifically, I would further filter the data on restaurants available through Foursquare and only include inner suburbs and only restaurants that are in some way comparable to the type of Mexican place that I would want to open. I.e., if it is a taco place, burger restaurants, fried chicken places and the like would seem to be the most comparable ones. I would also merge the Foursquare data with additional data on footfall, proximity to public transport and parking (some of which might be available through the City of Melbourne's open data portal). Additional exploratory analyses could be used to further limit the parameters that define a successful venue. E.g., after collecting a large number of parameters they could be regressed onto performance data for the comparable restaurants in the analysis (companies like Dun and Bradstreet might be able to offer revenue data, which would have to be normalized using, for example the seating capacity of the respective restaurant). Simple regression would allow to filter the success parameters in a way that retains the most telling ones, which could then be fed into the cluster analysis as additional clustering variables. Obviously, such an approach would include a lot of trial and error, which goes beyond the scope of this assignment, but I could imagine an application where the user specifies the type of restaurant to be opened, and the application spits out the top 10 comparable restaurants and their locations, plus a suggestion of similar locations which currently do not have a restaurant of the cuisine in question.

## Conclusion

For the purposes outlined in the introduction of this report, the application presented here is not quite sufficient to make an informed decision. Rather than only relying on Foursquare's data, a more sophisticated approach would have to include many additional data sources and would probably have a more fine-grained analysis than on the basis of suburbs - block or street level would be preferable. For the purpose of this assignemnt, however, the application provides an overview of the types of restaurants that are sought after in a given cluster of suburbs, which could be seen as a first step into the right direction.

Thanks for reading and enjoy the rest of your day!